# 5. Explicação da Previsão do Preço a partir da Base de Dados

## 5.1 Preparação dos dados

In [27]:
import pandas as pd

pd.set_option('display.max_columns', 100)

In [28]:
# carregando a base de dados de treinamento
df = pd.read_csv('/content/drive/MyDrive/Desafio Cientista de Dados - Indicium/cars_train.csv', encoding='UTF-16', delimiter='\t')

In [29]:
# verificando o numeros de linhas e colunas do dataframe
df.shape

(29584, 29)

In [42]:
df.dtypes

num_fotos              int64
marca                 object
modelo                object
versao                object
ano_de_fabricacao      int64
ano_modelo             int64
hodometro            float64
cambio                object
num_portas             int64
tipo                  object
blindado              object
cor                   object
tipo_vendedor         object
cidade_vendedor       object
estado_vendedor       object
anunciante            object
entrega_delivery        bool
troca                   bool
preco                float64
dtype: object

In [31]:
# configurando a indexação do dataframe pelo atributo id
df.set_index('id', inplace=True)

In [33]:
# excluindo os atributos irrelevantes para uso no modelo preditivo
colunas_para_excluir = ['veiculo_alienado', 'elegivel_revisao', 'dono_aceita_troca',
                                       'veiculo_único_dono', 'revisoes_concessionaria', 'ipva_pago',
                                       'veiculo_licenciado', 'garantia_de_fábrica', 'revisoes_dentro_agenda']
df = df.drop(columns=colunas_para_excluir)

In [37]:
# preenchendo os valores faltantes do atributo num_fotos com o valor mediano das amostras.
valor = df.describe()['num_fotos']['50%']
df['num_fotos'].fillna(valor, inplace=True)

In [39]:
# convertendo os valores do tipo float para int dos atributos num_fotos e ano_modelo
df['num_fotos'] = df['num_fotos'].astype(int)
df['ano_modelo'] = df['ano_modelo'].astype(int)

In [43]:
# convertendo os valores do tipo object para int do atributo blindado
df['blindado'] = df['blindado'].replace({'S': 1, 'N': 0})

In [44]:
# convertendo os valores do tipo bool para int dos atributos entrega_delivery e troca
df['entrega_delivery'] = df['entrega_delivery'].replace({True: 1, False: 0})
df['troca'] = df['troca'].replace({True: 1, False: 0})

In [48]:
# excluindo os atributos não considerados como atributos previsores para o modelo preditivo.
colunas_para_excluir = ['modelo', 'versao', 'cidade_vendedor', 'hodometro']
df = df.drop(columns=colunas_para_excluir)

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29584 entries, 300716223898539419613863097469899222392 to 171781413417552881896009715888472310172
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   num_fotos          29584 non-null  int64  
 1   marca              29584 non-null  object 
 2   ano_de_fabricacao  29584 non-null  int64  
 3   ano_modelo         29584 non-null  int64  
 4   cambio             29584 non-null  object 
 5   num_portas         29584 non-null  int64  
 6   tipo               29584 non-null  object 
 7   blindado           29584 non-null  int64  
 8   cor                29584 non-null  object 
 9   tipo_vendedor      29584 non-null  object 
 10  estado_vendedor    29584 non-null  object 
 11  anunciante         29584 non-null  object 
 12  entrega_delivery   29584 non-null  int64  
 13  troca              29584 non-null  int64  
 14  preco              29584 non-null  float64
dtypes: 

In [53]:
df.nunique()

num_fotos               14
marca                   40
ano_de_fabricacao       35
ano_modelo              17
cambio                   7
num_portas               3
tipo                     7
blindado                 2
cor                      7
tipo_vendedor            2
estado_vendedor         25
anunciante               4
entrega_delivery         2
troca                    2
preco                29584
dtype: int64

## 5.2 Seleção das features

In [59]:
def split_dataframe(df, target):
  X, y = df.drop(columns=[target], axis=1).values, df[target].values
  return X, y

In [61]:
# # separando a variável df em duas partes: variáveis X e y
X, y = split_dataframe(df, 'preco')
print(X.shape, type(X))
print(y.shape, type(y))

(29584, 14) <class 'numpy.ndarray'>
(29584,) <class 'numpy.ndarray'>


In [63]:
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder(sparse=False)

# aplicando a codificação one-hot na variável X
onehot_encoder.fit(X)
X_onehot = onehot_encoder.transform(X)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [66]:
X_onehot.shape

(29584, 167)

In [67]:
from sklearn.neighbors import LocalOutlierFactor

# identificando e eliminando de forma automatica outliers na base de dados de treinamento
lof = LocalOutlierFactor()
yhat = lof.fit_predict(X_onehot)

In [68]:
# selecionando todas as linhas que não são outliers
mask = yhat != -1
X_onehot, y = X_onehot[mask, :], y[mask]
print(X_onehot.shape, type(X_onehot))
print(y.shape, type(y))

(24260, 167) <class 'numpy.ndarray'>
(24260,) <class 'numpy.ndarray'>


In [71]:
# aplicando a normalização dos dados na variável y

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
y_norm = scaler.fit_transform(y.reshape(-1, 1))

In [72]:
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline

# criando um dicionario de modelos RFE
def get_models(num_features):
	models = dict()
	end = num_features//2
	for value in range(2, end):
		rfe = RFE(estimator=DecisionTreeRegressor(), n_features_to_select = value)
		model = DecisionTreeRegressor()
		models[str(value)] = Pipeline(steps=[('s', rfe), ('m',model)])
	return models

In [73]:
from sklearn.model_selection import cross_val_score, RepeatedKFold

# avaliando um determinado modelo usando validação cruzada
def evaluate_model(model, X, y):
	cv = RepeatedKFold(n_splits=5, n_repeats=1, random_state=1)
	scores = cross_val_score(model, X, y, scoring = 'neg_mean_squared_error', cv = cv, n_jobs = -1, error_score = 'raise')
	return scores

In [74]:
# criando os modelos RFE a serem avaliados por meio da validação cruizada
num_features = X_onehot.shape[1]
models = get_models(num_features)

In [75]:
import numpy as np

# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
	scores = evaluate_model(model, X_onehot, y_norm)
	results.append(scores)
	names.append(name)
	print(f'{name}: RMSE = {np.sqrt(np.abs(np.mean(scores))):.5f} ({np.sqrt(np.abs(np.std(scores))):.5f})')

2: RMSE = 0.05510 (0.01447)
3: RMSE = 0.05182 (0.01380)
4: RMSE = 0.05078 (0.01323)
5: RMSE = 0.04976 (0.01325)
6: RMSE = 0.04906 (0.01315)
7: RMSE = 0.04830 (0.01274)
8: RMSE = 0.04780 (0.01262)
9: RMSE = 0.04743 (0.01295)
10: RMSE = 0.04716 (0.01393)
11: RMSE = 0.04684 (0.01387)
12: RMSE = 0.04616 (0.01389)
13: RMSE = 0.04571 (0.01344)
14: RMSE = 0.04579 (0.01504)
15: RMSE = 0.04565 (0.01528)
16: RMSE = 0.04581 (0.01610)
17: RMSE = 0.04593 (0.01743)
18: RMSE = 0.04632 (0.01682)
19: RMSE = 0.04682 (0.01581)
20: RMSE = 0.04595 (0.01606)
21: RMSE = 0.04711 (0.01789)
22: RMSE = 0.04681 (0.01631)
23: RMSE = 0.04736 (0.01812)
24: RMSE = 0.04727 (0.01440)
25: RMSE = 0.04742 (0.01823)
26: RMSE = 0.04720 (0.01704)
27: RMSE = 0.04757 (0.01719)
28: RMSE = 0.04813 (0.01842)
29: RMSE = 0.04824 (0.01729)
30: RMSE = 0.04863 (0.01674)
31: RMSE = 0.04916 (0.01561)
32: RMSE = 0.04982 (0.01711)
33: RMSE = 0.04931 (0.01852)
34: RMSE = 0.04931 (0.01773)
35: RMSE = 0.05063 (0.01646)
36: RMSE = 0.05014 (0.

KeyboardInterrupt: ignored

In [76]:
# identificando as colunas selecionadas pelo método RFE
rfe = RFE(estimator = DecisionTreeRegressor(), n_features_to_select=15)
rfe.fit(X_onehot, y_norm)
for value in range(num_features):
	print(f'Column = {value}, Selected = {rfe.support_[value]}, Rank = {rfe.ranking_[value]:.2f}')

Column = 0, Selected = False, Rank = 11.00
Column = 1, Selected = False, Rank = 62.00
Column = 2, Selected = False, Rank = 153.00
Column = 3, Selected = False, Rank = 112.00
Column = 4, Selected = False, Rank = 76.00
Column = 5, Selected = False, Rank = 33.00
Column = 6, Selected = False, Rank = 28.00
Column = 7, Selected = False, Rank = 7.00
Column = 8, Selected = False, Rank = 22.00
Column = 9, Selected = False, Rank = 44.00
Column = 10, Selected = False, Rank = 68.00
Column = 11, Selected = False, Rank = 87.00
Column = 12, Selected = False, Rank = 48.00
Column = 13, Selected = False, Rank = 119.00
Column = 14, Selected = False, Rank = 129.00
Column = 15, Selected = False, Rank = 26.00
Column = 16, Selected = True, Rank = 1.00
Column = 17, Selected = False, Rank = 140.00
Column = 18, Selected = False, Rank = 79.00
Column = 19, Selected = False, Rank = 25.00
Column = 20, Selected = False, Rank = 106.00
Column = 21, Selected = False, Rank = 71.00
Column = 22, Selected = False, Rank = 1

In [78]:
# definindo as colunas selecionadas pelo método RFE como sendo as features
select_features = [16, 25, 34, 38, 43, 44, 86, 94, 95, 96, 97, 111, 120, 122, 125]
X_onehot_features = X_onehot[:, select_features]
print(X_onehot_features.shape)

(24260, 15)


## 5.3 Divisão dos Dados

In [79]:
from sklearn.model_selection import train_test_split

# Dividir o conjunto de dados em treinamento e teste (80% para treinamento, 20% para teste)
X_train, X_test, y_train_norm, y_test_norm = train_test_split(X_onehot_features , y_norm, test_size = 0.20, random_state=1)

## 5.4 Seleção e Treinamento do Modelo

In [80]:
!pip install autokeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.6/148.6 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 576.5/576.5 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 851.9/851.9 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 524.1/524.1 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.8/440.8 kB 8.2 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.7.1
    Uninstalling typing_extensions-4.7.1:
      Successfully uninstalled typing_extensions-4.7.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: ten

In [81]:
from autokeras import StructuredDataRegressor

# criando e treinando o modelo AutoKeras para a regressão
regressor = StructuredDataRegressor(max_trials=10, overwrite=True)
regressor.fit(X_train, y_train_norm, verbose = 0)

Using TensorFlow backend


## 5.5 Avaliação do Modelo Treinado

In [83]:
from sklearn.metrics import mean_squared_error
import numpy as np

# avaliando o modelo regressor com os dados de teste
mse, _ = regressor.evaluate(X_test,  y_test_norm, verbose=0)
rmse = np.sqrt(mse)
print(f'RMSE: {rmse:5f}')

RMSE: 0.044925


## 5.6 Precificando com o Modelo de melhor desempenho

In [84]:
# obtendo o modelo de melhor desempenho
model = regressor.export_model()

In [85]:
# resumindo a arquitetura e parametros do modelo de melhor desempenho
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 15)]              0         
                                                                 
 multi_category_encoding (M  (None, 15)                0         
 ultiCategoryEncoding)                                           
                                                                 
 normalization (Normalizati  (None, 15)                31        
 on)                                                             
                                                                 
 dense (Dense)               (None, 128)               2048      
                                                                 
 re_lu (ReLU)                (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 32)                4128  

In [95]:
# save the best performing model to file
model.save('/content/drive/MyDrive/Desafio Cientista de Dados - Indicium/model_auto', save_format='tf')

In [96]:
# carregando a base de dados de treinamento
df_test = pd.read_csv('/content/drive/MyDrive/Desafio Cientista de Dados - Indicium/cars_test.csv', encoding='UTF-16', delimiter='\t')

In [97]:
# verificando o numeros de linhas e colunas do dataframe
df_test.shape

(9862, 28)

In [98]:
df_test.isnull().sum()

id                            0
num_fotos                    60
marca                         0
modelo                        0
versao                        0
ano_de_fabricacao             0
ano_modelo                    0
hodometro                     0
cambio                        0
num_portas                    0
tipo                          0
blindado                      0
cor                           0
tipo_vendedor                 0
cidade_vendedor               0
estado_vendedor               0
anunciante                    0
entrega_delivery              0
troca                         0
elegivel_revisao              0
dono_aceita_troca          2554
veiculo_único_dono         6387
revisoes_concessionaria    6795
ipva_pago                  3298
veiculo_licenciado         4518
garantia_de_fábrica        8439
revisoes_dentro_agenda     7869
veiculo_alienado           9862
dtype: int64

In [99]:
df_test.dtypes

id                          object
num_fotos                  float64
marca                       object
modelo                      object
versao                      object
ano_de_fabricacao            int64
ano_modelo                 float64
hodometro                  float64
cambio                      object
num_portas                   int64
tipo                        object
blindado                    object
cor                         object
tipo_vendedor               object
cidade_vendedor             object
estado_vendedor             object
anunciante                  object
entrega_delivery              bool
troca                         bool
elegivel_revisao              bool
dono_aceita_troca           object
veiculo_único_dono          object
revisoes_concessionaria     object
ipva_pago                   object
veiculo_licenciado          object
garantia_de_fábrica         object
revisoes_dentro_agenda      object
veiculo_alienado           float64
dtype: object

In [100]:
# configurando a indexação do dataframe pelo atributo id
df_test.set_index('id', inplace=True)

In [101]:
# excluindo os atributos irrelevantes para uso no modelo preditivo
colunas_para_excluir = ['veiculo_alienado', 'elegivel_revisao', 'dono_aceita_troca',
                                       'veiculo_único_dono', 'revisoes_concessionaria', 'ipva_pago',
                                       'veiculo_licenciado', 'garantia_de_fábrica', 'revisoes_dentro_agenda']
df_test = df_test.drop(columns=colunas_para_excluir)

In [102]:
# preenchendo os valores faltantes do atributo num_fotos com o valor mediano das amostras.
valor = df_test.describe()['num_fotos']['50%']
df_test['num_fotos'].fillna(valor, inplace=True)

In [103]:
# convertendo os valores do tipo float para int dos atributos num_fotos e ano_modelo
df_test['num_fotos'] = df_test['num_fotos'].astype(int)
df_test['ano_modelo'] = df_test['ano_modelo'].astype(int)

In [104]:
# convertendo os valores do tipo object para int do atributo blindado
df_test['blindado'] = df_test['blindado'].replace({'S': 1, 'N': 0})

In [105]:
# convertendo os valores do tipo bool para int dos atributos entrega_delivery e troca
df_test['entrega_delivery'] = df_test['entrega_delivery'].replace({True: 1, False: 0})
df_test['troca'] = df_test['troca'].replace({True: 1, False: 0})

In [106]:
# excluindo os atributos não considerados como atributos previsores para o modelo preditivo.
colunas_para_excluir = ['modelo', 'versao', 'cidade_vendedor', 'hodometro']
df_test = df_test.drop(columns=colunas_para_excluir)

In [107]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9862 entries, 13518783164498355150900635905895481162 to 332784980027104367393384305973418103109
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   num_fotos          9862 non-null   int64 
 1   marca              9862 non-null   object
 2   ano_de_fabricacao  9862 non-null   int64 
 3   ano_modelo         9862 non-null   int64 
 4   cambio             9862 non-null   object
 5   num_portas         9862 non-null   int64 
 6   tipo               9862 non-null   object
 7   blindado           9862 non-null   int64 
 8   cor                9862 non-null   object
 9   tipo_vendedor      9862 non-null   object
 10  estado_vendedor    9862 non-null   object
 11  anunciante         9862 non-null   object
 12  entrega_delivery   9862 non-null   int64 
 13  troca              9862 non-null   int64 
dtypes: int64(7), object(7)
memory usage: 1.1+ MB


In [108]:
df_test.nunique()

num_fotos            13
marca                39
ano_de_fabricacao    34
ano_modelo           17
cambio                7
num_portas            3
tipo                  8
blindado              2
cor                   6
tipo_vendedor         2
estado_vendedor      24
anunciante            4
entrega_delivery      2
troca                 2
dtype: int64

In [110]:
# # separando a variável df em duas partes: variáveis X e y
X_test = df_test.values
print(X_test.shape, type(X_test))

(9862, 14) <class 'numpy.ndarray'>


In [111]:
# aplicando a codificação one-hot na variável X
onehot_encoder.fit(X_test)
X_test_onehot = onehot_encoder.transform(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [113]:
X_test_onehot.shape

(9862, 163)

In [114]:
X_test_onehot_features = X_test_onehot[:, select_features]
X_test_onehot_features.shape

(9862, 15)

In [116]:
# Fazer as previsões no conjunto de teste
y_predicted_norm = model.predict(X_test_onehot_features)

309/309 [==============================] - 1s 4ms/step


In [117]:
y_predicted = scaler.inverse_transform(y_predicted_norm)

In [119]:
print(y_predicted.shape, type(y_predicted))

(9862, 1) <class 'numpy.ndarray'>


In [123]:
id_predicted = df_test.index.to_numpy()
print(id_predicted.shape, type(id_predicted))

(9862,) <class 'numpy.ndarray'>


In [125]:
predicted_array = np.column_stack((id_predicted, y_predicted))

In [126]:
df_predicted = pd.DataFrame(predicted_array, columns=['id', 'preco'])

In [127]:
df_predicted.head()

,id,preco
0,13518783164498355150900635905895481162,130342.359375
1,299896161723793383246777788797566040330,130342.359375
2,316180649972302128246133616457018378621,274244.125
3,222527157104148385909188217274642813298,112290.882812
4,160460343059850745858546502614838368036,130342.359375


In [129]:
df_predicted.to_csv('/content/drive/MyDrive/Desafio Cientista de Dados - Indicium/predicted.csv', index=False)